In [35]:
%reload_ext autoreload
%autoreload 2

#load libraries
import glob
import os
import json
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun

import plot_funcs as pfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import pickle


# Get the data info
pfun.set_figure()

# Get the data info
infoPath = 'C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation/analysis/infoForAnalysis.pkl'
analysis_path = 'C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation/analysis/'
info = pd.read_pickle(infoPath)

## Parameters
fRate = 1000/30
responsiveness_test_duration = 1000.0 #in ms 
pre_frames    = 2000.0# in ms
pre_frames    = int(np.ceil(pre_frames/fRate))
post_frames   = 6000.0 # in ms
post_frames   = int(np.ceil(post_frames/fRate))
analysisWindowDur = 750 # in ms
analysisWindowDur = int(np.ceil(analysisWindowDur/fRate))

info.recordingList

,animalID,recordingDate,recordingID,sessionName,learningData,twoP,path,sessionNameWithPath,analysispathname,filepathname,eventTimesExtracted,eventTimesPath,imagingDataExtracted,imagingDataPath
0,MBL015,2025-05-14,1,2025-05-14_1_MBL015,False,True,Z:/MBL015\2025-05-14,Z:/MBL015\2025-05-14\1\2025-05-14_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-14\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...
1,MBL015,2025-05-15,1,2025-05-15_1_MBL015,False,True,Z:/MBL015\2025-05-15,Z:/MBL015\2025-05-15\1\2025-05-15_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-15\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...
2,MBL015,2025-05-16,1,2025-05-16_1_MBL015,False,True,Z:/MBL015\2025-05-16,Z:/MBL015\2025-05-16\1\2025-05-16_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-16\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...
3,MBL015,2025-05-20,1,2025-05-20_1_MBL015,False,True,Z:/MBL015\2025-05-20,Z:/MBL015\2025-05-20\1\2025-05-20_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-20\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...
4,MBL015,2025-05-22,1,2025-05-22_1_MBL015,False,True,Z:/MBL015\2025-05-22,Z:/MBL015\2025-05-22\1\2025-05-22_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-22\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...


In [36]:
# List of sessions to filter
# specific_expRefs = ['2025-05-22_1_MBL015', '2025-05-20_1_MBL015', '2025-05-16_1_MBL015', '2025-05-14_1_MBL015']
specific_expRefs = ['2025-05-22_1_MBL015']
# Filter the recording list to include only the specified sessions
info.recordingList = info.recordingList[info.recordingList['sessionName'].isin(specific_expRefs)].reset_index(drop=True)
info.recordingList

,animalID,recordingDate,recordingID,sessionName,learningData,twoP,path,sessionNameWithPath,analysispathname,filepathname,eventTimesExtracted,eventTimesPath,imagingDataExtracted,imagingDataPath
0,MBL015,2025-05-22,1,2025-05-22_1_MBL015,False,True,Z:/MBL015\2025-05-22,Z:/MBL015\2025-05-22\1\2025-05-22_1_MBL015_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/MBL015\2025-05-22\1,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,1.0,C:/Users/Lak Lab/Documents/Github/sideBiasLate...


In [ ]:
# Create JSON file for bias data

import json
import os

# Initialize JSON file with empty dict
bias_data = {}

# Make sure the folder exists
os.makedirs(analysis_path, exist_ok=True)

# Save the file in analysis folder
json_path = os.path.join(analysis_path, 'bias_data.json')
with open(json_path, 'w') as f:
    json.dump(bias_data, f)

# Read file from that folder
with open(json_path, 'r') as f:
    data = json.load(f)

In [ ]:
# Update bias data in JSON file
new_bias_data = {"MBL015": "Left"}
utils.update_json_data('bias_data.json', new_bias_data, analysis_path)

# Read JSON from analysis folder
with open(json_path, 'r') as f:
    bias_data = json.load(f)

bias_data

In [ ]:
# Create JSON file for recording side data

import json
import os

# Initialize JSON file with empty dict
bias_data = {}

# Make sure the folder exists
os.makedirs(analysis_path, exist_ok=True)

# Save the file in analysis folder
json_path = os.path.join(analysis_path, 'recside_data.json')
with open(json_path, 'w') as f:
    json.dump(bias_data, f)

# Read file from that folder
with open(json_path, 'r') as f:
    data = json.load(f)

In [ ]:
# Update recording side data in JSON file
new_recside_data = {"2025-05-22_1_MBL015": "Left", "2025-05-20_1_MBL015": "Right", "2025-05-16_1_MBL015": "Left", "2025-05-14_1_MBL015": "Right", "2025-05-15_1_MBL015": "Left"}

utils.update_json_data('recside_data.json', new_recside_data, analysis_path)

# Read JSON from analysis folder
with open(json_path, 'r') as f:
    recside_data = json.load(f)

recside_data

In [ ]:
# add ipsi and contra cols (to recording side and bias) to Correctedevents csv
utils.add_ipsi_contra_columns(info, analysis_path)

In [ ]:
# Explore dynamics in each sessions
# For each recording one file saved, each index is a cell 

# Option to use only responsive neurons
use_responsive_only = False  # Change to False to use all neurons

pd.set_option('mode.chained_assignment', None)

for ind, recordingDate in enumerate(info.recordingList.recordingDate):

   if (info.recordingList.imagingDataExtracted[ind]==1):
      print(str(ind) + ': Creating: ' + info.recordingList.analysispathname[ind])
      #Create a huge dictionary with all cells and parameters for each cell
      pathname = info.recordingList.analysispathname[ind]

      ########## Organise stimuli times 
      filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
      filenameCSV = [f for f in glob.glob(filenameCSV)]    
      behData     = pd.read_csv(filenameCSV[0], header=0)
      visTimes    = behData['stimulusOnsetTime']
      rewardTimes = behData['rewardTime']
      choice      = behData['choice']
      choiceTimes = behData['choiceStartTime']
      ipsi_contra_recside = behData['ipsi_contra_recside']

      # Create variable contrastTotal
      contrastTotal = behData['contrastLeft'] + behData['contrastRight']
      contrastTotal[ipsi_contra_recside == 'contra'] = -contrastTotal[ipsi_contra_recside == 'contra']

      # Make a variable for rewarded trials
      rewarded    =  behData['rewardTime'].notna() # True for rewarded which also means correct response
      # Calculate reward Time for non-rewarded Trials
      choiceCompleteTimes = behData['choiceCompleteTime']
      diff_time = np.nanmean(rewardTimes -choiceTimes)
      nan_indices = np.isnan(rewardTimes) # unrewarded trials
      rewardTimes[nan_indices] = choiceCompleteTimes[nan_indices] + diff_time
      
      # Create variable stimulus that takes the non-zero value of contrastLeft or contrastRight
      stimulus = np.where(behData['contrastLeft'] != 0, behData['contrastLeft'], behData['contrastRight'])

      # Make a variable for stimType
      stimType = pd.Series([''] * len(behData['choice']))
      stimType [:] = 'Left'

      # Stimulus type Right
      idx_right = behData['correctResponse'] == 'Right'
      stimType[idx_right] = 'Right'
      
      # Stimulus type Zero
      idx_zero = behData['contrastRight'] - behData['contrastLeft'] == 0
      stimType[idx_zero] = 'Zero'

      # Get the stim start times 
      filenameTXT = os.path.join(info.recordingList.path[ind],'twoP') +'\*_imaging_frames.txt'
      filenameTXT= [f for f in glob.glob(filenameTXT)]    
      frame_clock = pd.read_csv(filenameTXT[0],  header= None)
         
      stimFrameTimes    = utils.stim_start_frame_Dual2Psetup (frame_clock, visTimes)
      rewardFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, rewardTimes)
      choiceFrameTimes  = utils.stim_start_frame_Dual2Psetup (frame_clock, choiceTimes)

      ########## Organise calcium imaging traces 
      imData = pd.read_pickle (pathname +'imaging-data.pkl')
      fluR      = imData['flu']
      stat      = imData['stat']

      # clean detrended traces
      flu = utils.clean_traces(fluR)

      # Z-score for each neuron (across time)
      flu_zscored = zscore(flu, axis=1)

      #flu_normalised = mfun.norm_to_zero_one (flu)
      flu_normalised = zscore (fluR)
      dffTrace_reward ={} 
      dffTrace_mean_reward ={}
      dffTrace_stimuli ={} 
      dffTrace_mean_stimuli ={}
      dffTrace_choice ={}
      dffTrace_mean_choice ={}
      dff_mean_stimuli ={}
      dff_mean_reward ={}
      dff_mean_choice ={}

      ### Get dff values for 2 time windows
      tTypesName = ['Rewarded', 'Unrewarded',              # reward type
                  'Left choices', 'Right choices',       # choice type
                  'Rewarded Left', 'Rewarded Right',     # new combined types
                  '0.0625','0.125', '0.25', '0.5',       # stimuli contrast type
                  'ipsi_recside', 'contra_recside',        # stimuli recording side type
                  'ipsi 0.0625','ipsi 0.125', 'ipsi 0.25', 'ipsi 0.5',       # ipsi contrastTotal type
                  'contra 0.0625','contra 0.125', 'contra 0.25', 'contra 0.5',       # contra contrastTotal type
                  'zero contrast'             # contrast 0
                  ]
      tTypes = [(rewarded==True),(rewarded!=True),          # reward type
               (choice == 'Left'),(choice == 'Right'),      # choice type
               (rewarded==True) & (choice == 'Left'),       # new combined type
               (rewarded==True) & (choice == 'Right'),      # new combined type
               (stimulus == 0.0625),(stimulus == 0.125), 
               (stimulus == 0.25),(stimulus == 0.5),        # stimuli contrast type
               (ipsi_contra_recside == 'ipsi'),              # stimuli side type
               (ipsi_contra_recside == 'contra'),             # stimuli side type
               (contrastTotal == 0.0625),(contrastTotal == 0.125), # contrastTotal type
               (contrastTotal == 0.25),(contrastTotal == 0.5),  # contrastTotal type
               (contrastTotal == -0.0625),(contrastTotal == -0.125), # contrastTotal type
               (contrastTotal == -0.25),(contrastTotal == -0.5), # contrastTotal type
               (contrastTotal == 0)
               ]


      for indx, t in enumerate(tTypesName):
         # For reward
         selected_indices = [rewardFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(rewardFrameTimes)))[0]]  
         selected_indices = [value for value in selected_indices if value == value]
         dffTrace_reward[t] = utils.flu_splitter(flu,selected_indices, pre_frames, post_frames)  # Cell x time x trial
         dffTrace_mean_reward[t] = np.mean(dffTrace_reward[t],2) if len(selected_indices)>2 else None # Cell x time
         dff_mean_reward[t] = np.nanmean(dffTrace_mean_reward[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None
         
         # For stimuli
         selected_indices = [stimFrameTimes[i] for i in np.where(tTypes[indx]==True)[0]]
         selected_indices = [value for value in selected_indices if value == value]
         dffTrace_stimuli[t] = utils.flu_splitter(flu, selected_indices, pre_frames, post_frames) # Cell x time x trial
         dffTrace_mean_stimuli[t] = np.mean(dffTrace_stimuli[t],2) if len(selected_indices)>2 else None # Cell x time
         dff_mean_stimuli[t] = np.nanmean(dffTrace_mean_stimuli[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

         # For choice
         selected_indices = [choiceFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(choiceFrameTimes)))[0]] 
         selected_indices = [value for value in selected_indices if value == value]
         dffTrace_choice[t] = utils.flu_splitter(flu, selected_indices, pre_frames, post_frames) # Cell x time x trial
         dffTrace_mean_choice[t] = np.mean(dffTrace_choice[t],2) if len(selected_indices)>2 else None # Cell x time
         dff_mean_choice[t] = np.nanmean(dffTrace_mean_choice[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

      # --- NEW BLOCK FOR Z-SCORED ---
         dffTrace_reward_z = {}
         dffTrace_mean_reward_z = {}
         dffTrace_stimuli_z = {}
         dffTrace_mean_stimuli_z = {}
         dffTrace_choice_z = {}
         dffTrace_mean_choice_z = {}
         dff_mean_stimuli_z = {}
         dff_mean_reward_z = {}
         dff_mean_choice_z = {}

         for indx, t in enumerate(tTypesName):
            # Reward
            selected_indices = [rewardFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(rewardFrameTimes)))[0]]  
            selected_indices = [value for value in selected_indices if value == value]
            dffTrace_reward_z[t] = utils.flu_splitter(flu_zscored, selected_indices, pre_frames, post_frames)
            dffTrace_mean_reward_z[t] = np.mean(dffTrace_reward_z[t],2) if len(selected_indices)>2 else None
            dff_mean_reward_z[t] = np.nanmean(dffTrace_mean_reward_z[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

            # Stimuli
            selected_indices = [stimFrameTimes[i] for i in np.where(tTypes[indx]==True)[0]]
            selected_indices = [value for value in selected_indices if value == value]
            dffTrace_stimuli_z[t] = utils.flu_splitter(flu_zscored, selected_indices, pre_frames, post_frames)
            dffTrace_mean_stimuli_z[t] = np.mean(dffTrace_stimuli_z[t],2) if len(selected_indices)>2 else None
            dff_mean_stimuli_z[t] = np.nanmean(dffTrace_mean_stimuli_z[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None

            # Choice
            selected_indices = [choiceFrameTimes[i] for i in np.where((tTypes[indx]==True)& (~np.isnan(choiceFrameTimes)))[0]] 
            selected_indices = [value for value in selected_indices if value == value]
            dffTrace_choice_z[t] = utils.flu_splitter(flu_zscored, selected_indices, pre_frames, post_frames)
            dffTrace_mean_choice_z[t] = np.mean(dffTrace_choice_z[t],2) if len(selected_indices)>2 else None
            dff_mean_choice_z[t] = np.nanmean(dffTrace_mean_choice_z[t][:, (pre_frames): (pre_frames + analysisWindowDur)],1) if len(selected_indices)>2 else None


      ##################
      # Filter significantly responsive neurons for each alignment type
      if use_responsive_only:

         # For stimuli
         responsive_neurons_stimuli = utils.filter_responsive_neurons(dffTrace_mean_stimuli, pre_frames, post_frames)

         # For choice
         responsive_neurons_choice = utils.filter_responsive_neurons(dffTrace_mean_choice, pre_frames, post_frames)

         # For reward
         responsive_neurons_reward = utils.filter_responsive_neurons(dffTrace_mean_reward, pre_frames, post_frames)

         # Filter significantly responsive neurons for each alignment type
         print('Filtering significantly responsive neurons...')
         
         # Filter traces to use only responsive neurons
         for condition in dffTrace_mean_stimuli.keys():
            if dffTrace_mean_stimuli[condition] is not None and responsive_neurons_stimuli[condition] is not None:
                  # Filter original data
                  dffTrace_stimuli[condition] = dffTrace_stimuli[condition][responsive_neurons_stimuli[condition]]
                  dffTrace_mean_stimuli[condition] = dffTrace_mean_stimuli[condition][responsive_neurons_stimuli[condition]]
                  dff_mean_stimuli[condition] = dff_mean_stimuli[condition][responsive_neurons_stimuli[condition]]
                  # Filter z-scored data
                  dffTrace_stimuli_z[condition] = dffTrace_stimuli_z[condition][responsive_neurons_stimuli[condition]]
                  dffTrace_mean_stimuli_z[condition] = dffTrace_mean_stimuli_z[condition][responsive_neurons_stimuli[condition]]
                  dff_mean_stimuli_z[condition] = dff_mean_stimuli_z[condition][responsive_neurons_stimuli[condition]]
         
         for condition in dffTrace_mean_choice.keys():
            if dffTrace_mean_choice[condition] is not None and responsive_neurons_choice[condition] is not None:
                  # Filter original data
                  dffTrace_choice[condition] = dffTrace_choice[condition][responsive_neurons_choice[condition]]
                  dffTrace_mean_choice[condition] = dffTrace_mean_choice[condition][responsive_neurons_choice[condition]]
                  dff_mean_choice[condition] = dff_mean_choice[condition][responsive_neurons_choice[condition]]
                  # Filter z-scored data
                  dffTrace_choice_z[condition] = dffTrace_choice_z[condition][responsive_neurons_choice[condition]]
                  dffTrace_mean_choice_z[condition] = dffTrace_mean_choice_z[condition][responsive_neurons_choice[condition]]
                  dff_mean_choice_z[condition] = dff_mean_choice_z[condition][responsive_neurons_choice[condition]]
         
         for condition in dffTrace_mean_reward.keys():
            if dffTrace_mean_reward[condition] is not None and responsive_neurons_reward[condition] is not None:
                  # Filter original data
                  dffTrace_reward[condition] = dffTrace_reward[condition][responsive_neurons_reward[condition]]
                  dffTrace_mean_reward[condition] = dffTrace_mean_reward[condition][responsive_neurons_reward[condition]]
                  dff_mean_reward[condition] = dff_mean_reward[condition][responsive_neurons_reward[condition]]
                  # Filter z-scored data
                  dffTrace_reward_z[condition] = dffTrace_reward_z[condition][responsive_neurons_reward[condition]]
                  dffTrace_mean_reward_z[condition] = dffTrace_mean_reward_z[condition][responsive_neurons_reward[condition]]
                  dff_mean_reward_z[condition] = dff_mean_reward_z[condition][responsive_neurons_reward[condition]]
         
         print(f'Using only significantly responsive neurons')
         print(f'Number of neurons per condition:')
         print('Stimuli:')
         for condition, traces in dffTrace_mean_stimuli.items():
            if traces is not None:
                  print(f'{condition}: {traces.shape[0]} neurons')
         print('Choice:')
         for condition, traces in dffTrace_mean_choice.items():
            if traces is not None:
                  print(f'{condition}: {traces.shape[0]} neurons')
         print('Reward:')
         for condition, traces in dffTrace_mean_reward.items():
            if traces is not None:
                  print(f'{condition}: {traces.shape[0]} neurons')
      ###################

      # Determine subfolder based on use_responsive_only
      subfolder = 'responsive_neurons' if use_responsive_only else 'all_neurons'
      subfolder_path = os.path.join(pathname, subfolder)
      os.makedirs(subfolder_path, exist_ok=True)

      # Save params in the appropriate subfolder
      filenameINFO = os.path.join(subfolder_path, 'imaging-dffTrace.pkl')
      print('Saving: '+ filenameINFO)
      with open(filenameINFO, 'wb') as f:
         pickle.dump([dffTrace_reward,dffTrace_stimuli, dffTrace_choice] , f)

      filenameINFO = os.path.join(subfolder_path, 'imaging-dffTrace_mean.pkl')
      print('Saving: '+ filenameINFO)
      with open(filenameINFO, 'wb') as f:
         pickle.dump([dffTrace_mean_reward,
                     dffTrace_mean_stimuli,
                     dffTrace_mean_choice ] , f)

      filenameINFO = os.path.join(subfolder_path, 'imaging-dff_mean.pkl')
      print('Saving: '+ filenameINFO)
      with open(filenameINFO, 'wb') as f:
         pickle.dump([dff_mean_reward,
                     dff_mean_stimuli,
                     dff_mean_choice ] , f)

      # Save z-scored data in the same subfolder
      filenameINFO = os.path.join(subfolder_path, 'imaging-dffTrace_zscored.pkl')
      print('Saving: '+ filenameINFO)
      with open(filenameINFO, 'wb') as f:
         pickle.dump([dffTrace_reward_z, dffTrace_stimuli_z, dffTrace_choice_z], f)

      filenameINFO = os.path.join(subfolder_path, 'imaging-dffTrace_mean_zscored.pkl')
      print('Saving: '+ filenameINFO)
      with open(filenameINFO, 'wb') as f:
         pickle.dump([dffTrace_mean_reward_z, dffTrace_mean_stimuli_z, dffTrace_mean_choice_z], f)

      filenameINFO = os.path.join(subfolder_path, 'imaging-dff_mean_zscored.pkl')
      print('Saving: '+ filenameINFO)
      with open(filenameINFO, 'wb') as f:
         pickle.dump([dff_mean_reward_z, dff_mean_stimuli_z, dff_mean_choice_z], f)

      # Modify the plotting functions to save in the subfolder
      # Heat Plots
      savefigname = 'SessionComparison-heatmap-reward_' + str(duration[0]) + 'sec'
      analysis_params = ['Rewarded', 'Unrewarded']
      pfun.heatmap_sessions(dffTrace_mean_reward, analysis_params, colormap,
                           selectedSession, duration, savefigname, subfolder_path) 
      plt.close()
                     
      savefigname = 'SessionComparison-heatmap-choice_' + str(duration[0]) + 'sec'
      analysis_params = ['Left choices','Right choices']
      pfun.heatmap_sessions(dffTrace_mean_choice, analysis_params, colormap,
                           selectedSession, duration, savefigname, subfolder_path) 
      plt.close()

      savefigname = 'SessionComparison-heatmap-stimuli_' + str(duration[0]) + 'sec'
      analysis_params = ['0.0625','0.125', '0.25', '0.5']
      pfun.heatmap_sessions(dffTrace_mean_stimuli, analysis_params, colormap,
                           selectedSession, duration, savefigname, subfolder_path) 
      plt.close()

      savefigname = 'SessionComparison-heatmap-stimuli_recside_' + str(duration[0]) + 'sec'
      analysis_params = ['ipsi_recside', 'contra_recside']
      pfun.heatmap_sessions(dffTrace_mean_stimuli, analysis_params, colormap,
                           selectedSession, duration, savefigname, subfolder_path) 
      plt.close()

      # Mean Plots 
      colormap = 'Set2'
      zscoreRun = False 

      savefigname = 'SessionComparison-mean-reward_' + str(duration[0]) + 'sec'
      analysis_params = ['Rewarded', 'Unrewarded']
      pfun.lineplot_sessions(dffTrace_mean_reward, analysis_params, colormap,
                           duration, zscoreRun, savefigname, subfolder_path) 
      plt.close()

      savefigname = 'SessionComparison-mean-choice_' + str(duration[0]) + 'sec'
      analysis_params = ['Left choices','Right choices']
      pfun.lineplot_sessions(dffTrace_mean_choice, analysis_params, colormap,
                           duration, zscoreRun, savefigname, subfolder_path) 
      plt.close()

      savefigname = 'SessionComparison-mean-choice-ipsi-contra_' + str(duration[0]) + 'sec'
      analysis_params = ['ipsi_recside', 'contra_recside']
      pfun.lineplot_sessions(dffTrace_mean_choice, analysis_params, colormap,
                           duration, zscoreRun, savefigname, subfolder_path) 
      plt.close()

      savefigname = 'SessionComparison-mean-stimuli-ipsi-contra_' + str(duration[0]) + 'sec'
      analysis_params = ['ipsi_recside', 'contra_recside']
      pfun.lineplot_sessions(dffTrace_mean_stimuli, analysis_params, colormap,
                           duration, zscoreRun, savefigname, subfolder_path) 
      plt.close()

      print('Completed')
      info.recordingList.loc[ind,'analysisVariableExtracted'] = 0 


0: Creating: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons\imaging-dffTrace.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons\imaging-dffTrace_mean.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons\imaging-dff_mean.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons\imaging-dffTrace_zscored.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons\imaging-dffTrace_mean_zscored.pkl
Saving: C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons\imaging-dff_mean_zscored.pkl
Completed


In [42]:
# Plot dff_mean_zscored by contrast
# negative contrasts are contra, positive contrasts are ipsi
use_responsive_only = False  # Cambiar a False para usar todas las neuronas

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    if info.recordingList.imagingDataExtracted[ind] == 1:
        session_path = info.recordingList.analysispathname[ind]
        print(f'Processing session: {info.recordingList.sessionName[ind]}')
        try:
            # Determinar la subcarpeta basada en use_responsive_only
            subfolder = 'responsive_neurons' if use_responsive_only else 'all_neurons'
            subfolder_path = os.path.join(session_path, subfolder)
            os.makedirs(subfolder_path, exist_ok=True)
            
            # Llamar a la función con todos los parámetros necesarios
            utils.plot_dff_mean_by_contrast(
                session_path=session_path,
                save_path=subfolder_path,
                use_responsive_only=use_responsive_only
            )
            print(f'Plot saved in {subfolder_path} for {info.recordingList.sessionName[ind]}')
        except Exception as e:
            print(f'Error processing {info.recordingList.sessionName[ind]}: {str(e)}')

Processing session: 2025-05-22_1_MBL015
Plot saved in C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-05-22_MBL015_1\all_neurons for 2025-05-22_1_MBL015
